In [1]:
import sqlite3

import numpy as np
import pandas as pd
from faker import Faker


def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

## degrees 

In [2]:
conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
df


,StudentID,Name,Degree,Exams,Scores
0,1,"Rodriguez, Pamela",graduate,"exam7 (2017), exam9 (2018), exam3 (2018), exam...","61, 38, 85, 70, 44, 43, 68"
1,2,"Jackson, Kristie",undergraduate,"exam2 (2017), exam2 (2017), exam8 (2018), exam...","66, 69, 85, 62, 92, 72, 44, 50, 23"
2,3,"Curtis, George",graduate,"exam7 (2017), exam6 (2017), exam2 (2017), exam...","68, 73, 62, 72"
3,4,"Casey, Christie",undergraduate,"exam10 (2019), exam4 (2019), exam4 (2019), exa...","41, 52, 52, 70"
4,5,"Yoder, Emily",undergraduate,"exam4 (2019), exam4 (2019), exam4 (2019)","52, 45, 61"
...,...,...,...,...,...
95,96,"Dyer, Benjamin",undergraduate,"exam10 (2019), exam4 (2019), exam6 (2017), exa...","47, 53, 64, 57, 71, 79, 77, 56"
96,97,"Delgado, Jason",undergraduate,"exam1 (2016), exam8 (2018)","72, 83"
97,98,"Cortez, Kenneth",undergraduate,"exam3 (2018), exam5 (2020), exam6 (2017)","83, 55, 71"
98,99,"Short, Elizabeth",undergraduate,"exam3 (2018), exam9 (2018), exam3 (2018), exam...","87, 34, 84, 75, 80, 60, 48, 55, 75"


## create_df_exams

In [3]:
conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)

exam_year = []
for index, row in df.iterrows():
    series_maal = row.values
    exams = [tuple(i.split(" ")) for i in series_maal[3].split(', ')]
    exam_year.extend([(exam, int(year[1:-1])) for exam, year in exams])

df2 = pd.DataFrame(exam_year, columns=['Exam', 'Year'])
df2 = df2.sort_values(by = ['Exam']).reset_index(drop = True)

df_exams = df2.drop_duplicates().reset_index(drop = True)


df_exams

,Exam,Year
0,exam1,2016
1,exam10,2019
2,exam2,2017
3,exam3,2018
4,exam4,2019
5,exam5,2020
6,exam6,2017
7,exam7,2017
8,exam8,2018
9,exam9,2018


In [4]:
conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
exam_list = df["Exams"].unique()

exam_name = []
exam_date = []
exam_hasho = {}
# for i in range(len(exam_list)):
#     temp = exam_list[i].strip().split(",")
#     for i in range(len(temp)):
#         if temp[i].strip().split(" ")[0] not in exam_name:
#             if temp[i].strip().split(" ")[0] != " ":
#                 exam_name.append(temp[i].strip().split(" ")[0])
#         if temp[i].strip().split(" ")[1] not in exam_date:
#             if temp[i].strip().split(" ")[1] != " ":
#                 exam_date.append(temp[i].strip().split(" ")[1])
# exam_date

unique_exam_and_date = []
for i in range(len(exam_list)):
    exam_and_date = exam_list[i].split(",")
    for i in range(len(exam_and_date)):
        if exam_and_date[i].strip() not in unique_exam_and_date:
            unique_exam_and_date.append(exam_and_date[i].strip())

exam_hasho = {"Exam": [], "Year": []}

for exam in unique_exam_and_date:
    exam_name, year = exam.split(" (")
    exam_hasho["Exam"].append(exam_name)
    exam_hasho["Year"].append(year[:-1])     
df_exams = pd.DataFrame(exam_hasho)
df_exams
    
    
    

,Exam,Year
0,exam7,2017
1,exam9,2018
2,exam3,2018
3,exam5,2020
4,exam10,2019
5,exam1,2016
6,exam2,2017
7,exam8,2018
8,exam6,2017
9,exam4,2019


In [5]:
unique_exam_and_date.sort()

In [6]:
exam_hasho = {"Exam": [], "Year": []}

for exam in unique_exam_and_date:
    exam_name, year = exam.split(" (")
    exam_hasho["Exam"].append(exam_name)
    exam_hasho["Year"].append(year[:-1])

exam_hasho

{'Exam': ['exam1',
  'exam10',
  'exam2',
  'exam3',
  'exam4',
  'exam5',
  'exam6',
  'exam7',
  'exam8',
  'exam9'],
 'Year': ['2016',
  '2019',
  '2017',
  '2018',
  '2019',
  '2020',
  '2017',
  '2017',
  '2018',
  '2018']}

In [7]:
exams_hasho = { 
               "Exam" : [], 
               "Year" : []
               }

for i in range(len(unique_exam_and_date)):
    exam_hasho["Exam"].append(unique_exam_and_date[i].split(" ")[0])
    exam_hasho["Year"].append(unique_exam_and_date[i].split(" ")[1].replace("(", "").replace(")", ""))
exam_hasho   

{'Exam': ['exam1',
  'exam10',
  'exam2',
  'exam3',
  'exam4',
  'exam5',
  'exam6',
  'exam7',
  'exam8',
  'exam9',
  'exam1',
  'exam10',
  'exam2',
  'exam3',
  'exam4',
  'exam5',
  'exam6',
  'exam7',
  'exam8',
  'exam9'],
 'Year': ['2016',
  '2019',
  '2017',
  '2018',
  '2019',
  '2020',
  '2017',
  '2017',
  '2018',
  '2018',
  '2016',
  '2019',
  '2017',
  '2018',
  '2019',
  '2020',
  '2017',
  '2017',
  '2018',
  '2018']}

In [8]:
# for exam_year in exam_list[0].split(","):
#     for i in range(len(exam_year)):
#         print(exam_year.split(" "))
# temp = exam_list[34].strip().split(",")
# for i in range(len(temp)):
#     print(temp[i].strip().split(" ")[0])
exam_list[0].split(",")

['exam7 (2017)',
 ' exam9 (2018)',
 ' exam3 (2018)',
 ' exam7 (2017)',
 ' exam5 (2020)',
 ' exam10 (2019)',
 ' exam1 (2016)']

In [9]:

exam_list= []
unique_exam_and_date = []

conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
exam_list = df["Exams"].unique()

exam_hasho = {"Exam": [], "Year": []}
for i in range(len(exam_list)):
    exam_and_date = exam_list[i].split(",")
    for i in range(len(exam_and_date)):
        if exam_and_date[i].strip() not in unique_exam_and_date:
            unique_exam_and_date.append(exam_and_date[i].strip())
unique_exam_and_date.sort()


for exam in unique_exam_and_date:
    exam_name, year = exam.split(" (")
    exam_hasho["Exam"].append(exam_name)
    exam_hasho["Year"].append(year[:-1])     

df_2 = pd.DataFrame.from_dict(exam_hasho)
df_2

,Exam,Year
0,exam1,2016
1,exam10,2019
2,exam2,2017
3,exam3,2018
4,exam4,2019
5,exam5,2020
6,exam6,2017
7,exam7,2017
8,exam8,2018
9,exam9,2018


In [10]:
exam_list= []
unique_exam_and_date = []

conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
exam_list = df["Exams"].unique()

exam_hasho = {"Exam": [], "Year": []}
for i in range(len(exam_list)):
    exam_and_date = exam_list[i].split(",")
    for i in range(len(exam_and_date)):
        if exam_and_date[i].strip() not in unique_exam_and_date:
            unique_exam_and_date.append(exam_and_date[i].strip())
unique_exam_and_date.sort()


for exam in unique_exam_and_date:
    exam_name, year = exam.split(" (")
    exam_hasho["Exam"].append(exam_name)
    exam_hasho["Year"].append(int(year[:-1]))     
df_2 = pd.DataFrame.from_dict(exam_hasho)
df_2

,Exam,Year
0,exam1,2016
1,exam10,2019
2,exam2,2017
3,exam3,2018
4,exam4,2019
5,exam5,2020
6,exam6,2017
7,exam7,2017
8,exam8,2018
9,exam9,2018


In [11]:
df_exams_from_file = pd.read_csv("df_exams.csv")
df_exams_from_file

,Exam,Year
0,exam1,2016
1,exam10,2019
2,exam2,2017
3,exam3,2018
4,exam4,2019
5,exam5,2020
6,exam6,2017
7,exam7,2017
8,exam8,2018
9,exam9,2018


## create_df_students

In [12]:
conn = create_connection("non_normalized.db")
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)


student_names = df["Name"]
student_degrees = df["Degree"]


students_hasho = {"StudentID": [], "First_Name": [], "Last_Name" : [], "Degree": []}


for i in range(len(student_names)):
    students_hasho["StudentID"].append(i+1)
    students_hasho["First_Name"].append(student_names[i].split(", ")[1])
    students_hasho["Last_Name"].append(student_names[i].split(", ")[0])
    students_hasho["Degree"].append(student_degrees[i])
    
df_students = pd.DataFrame.from_dict(students_hasho)
df_students

,StudentID,First_Name,Last_Name,Degree
0,1,Pamela,Rodriguez,graduate
1,2,Kristie,Jackson,undergraduate
2,3,George,Curtis,graduate
3,4,Christie,Casey,undergraduate
4,5,Emily,Yoder,undergraduate
...,...,...,...,...
95,96,Benjamin,Dyer,undergraduate
96,97,Jason,Delgado,undergraduate
97,98,Kenneth,Cortez,undergraduate
98,99,Elizabeth,Short,undergraduate


In [13]:

student_degrees

0          graduate
1     undergraduate
2          graduate
3     undergraduate
4     undergraduate
          ...      
95    undergraduate
96    undergraduate
97    undergraduate
98    undergraduate
99    undergraduate
Name: Degree, Length: 100, dtype: object

## create_df_studentexamscores

In [14]:
conn = create_connection("non_normalized.db")
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)

student_exams = df["Exams"]
student_scores = df["Scores"]
student_ids = df["StudentID"]

exam_scores_hasho = {
    "StudentID" : [],
    "Exam" : [],
    "Score" : []
}

for i in range(len(student_exams)):
    ith_student_exams = student_exams[i].split(" ")[::2]
    ith_student_scores = student_scores[i].split(", ")
    for j in range(len(ith_student_exams)):
        exam_scores_hasho["StudentID"].append(i+1)
        exam_scores_hasho['Exam'].append(ith_student_exams[j])
        exam_scores_hasho["Score"].append(int(ith_student_scores[j]))

df_studentexamscores = pd.DataFrame.from_dict(exam_scores_hasho)
df_studentexamscores
        

,StudentID,Exam,Score
0,1,exam7,61
1,1,exam9,38
2,1,exam3,85
3,1,exam7,70
4,1,exam5,44
...,...,...,...
579,99,exam4,60
580,99,exam10,48
581,99,exam4,55
582,99,exam7,75


In [15]:
ith_student_exams = student_exams[0].split(" ")[::2]

In [16]:
ith_student_scores =( student_scores[0].split(", "))
ith_student_scores

['61', '38', '85', '70', '44', '43', '68']

## ex 1

In [17]:
df_exams

,Exam,Year
0,exam7,2017
1,exam9,2018
2,exam3,2018
3,exam5,2020
4,exam10,2019
5,exam1,2016
6,exam2,2017
7,exam8,2018
8,exam6,2017
9,exam4,2019


In [18]:
df_exams.sort_values(by='Year')


,Exam,Year
5,exam1,2016
0,exam7,2017
6,exam2,2017
8,exam6,2017
1,exam9,2018
2,exam3,2018
7,exam8,2018
4,exam10,2019
9,exam4,2019
3,exam5,2020


## ex2

In [19]:
df_students

,StudentID,First_Name,Last_Name,Degree
0,1,Pamela,Rodriguez,graduate
1,2,Kristie,Jackson,undergraduate
2,3,George,Curtis,graduate
3,4,Christie,Casey,undergraduate
4,5,Emily,Yoder,undergraduate
...,...,...,...,...
95,96,Benjamin,Dyer,undergraduate
96,97,Jason,Delgado,undergraduate
97,98,Kenneth,Cortez,undergraduate
98,99,Elizabeth,Short,undergraduate


In [20]:
df_degree_count = df_students['Degree'].value_counts().reset_index()
df_degree_count = df_degree_count.rename(columns={'index':'Count'})
df_degree_count

,Count,Degree
0,undergraduate,65
1,graduate,35


In [21]:
# df.columns = pd.MultiIndex.from_frame(df)
# df = pd.MultiIndex.from_frame(df)
df
df.set_index(0,inplace=True)

df = df_students["Degree"].value_counts().to_frame()
df.columns=[['Count']]



KeyError: 'None of [0] are in the columns'

In [ ]:
df_file = pd.read_csv("ex2.csv", index_col=0)
df_file.columns = [['Count']]
df_file 

,Count
undergraduate,65
graduate,35


In [ ]:
from pandas.testing import assert_frame_equal

assert_frame_equal(df, df_file)

AssertionError: DataFrame Expected type <class 'pandas.core.frame.DataFrame'>, found <class 'pandas.core.indexes.multi.MultiIndex'> instead

In [ ]:
df = pd.MultiIndex(df)


TypeError: Must pass both levels and codes

In [ ]:
df_studentexamscores

,StudentID,Exam,Score
0,1,exam7,61
1,1,exam9,38
2,1,exam3,85
3,1,exam7,70
4,1,exam5,44
...,...,...,...
579,99,exam4,60
580,99,exam10,48
581,99,exam4,55
582,99,exam7,75


## ex 3

In [ ]:
# Merge df_studentexamscores and df_exams
df = pd.merge(df_studentexamscores, df_exams, on='Exam')

# Group by student and exam, and compute the mean score
df_averages = df.groupby(['Exam', 'Year'])['Score'].mean().round(2).reset_index()
# Sort the dataframe in descending order by the average score
df_averages.sort_values(by='Score', ascending=False, inplace=True)
df_averages["Score"].astype("int32")
df_averages["Year"] = df_averages["Year"].astype("int32")

df_averages = df_averages.rename(columns={'Score': 'average'})
df_averages.set_index('Exam', inplace=True)
df_averages

,Year,average
Exam,,
exam3,2018,86.16
exam8,2018,80.18
exam1,2016,75.09
exam7,2017,68.96
exam2,2017,64.28
exam6,2017,61.80
exam4,2019,54.04
exam5,2020,45.56
exam10,2019,44.68


In [ ]:
df_from_file = pd.read_csv("ex3.csv", index_col=0)
df_from_file['Year'] = df_from_file['Year'].astype('int32')
df_from_file

,Year,average
Exam,,
exam3,2018,86.16
exam8,2018,80.18
exam1,2016,75.09
exam7,2017,68.96
exam2,2017,64.28
exam6,2017,61.80
exam4,2019,54.04
exam5,2020,45.56
exam10,2019,44.68


In [ ]:
from pandas.testing import assert_frame_equal

assert_frame_equal(df_averages, df_from_file)

## Ex 4

In [ ]:
df_studentexamscores

,StudentID,Exam,Score
0,1,exam7,61
1,1,exam9,38
2,1,exam3,85
3,1,exam7,70
4,1,exam5,44
...,...,...,...
579,99,exam4,60
580,99,exam10,48
581,99,exam4,55
582,99,exam7,75


In [ ]:
df_studentexamscores

,StudentID,Exam,Score
0,1,exam7,61
1,1,exam9,38
2,1,exam3,85
3,1,exam7,70
4,1,exam5,44
...,...,...,...
579,99,exam4,60
580,99,exam10,48
581,99,exam4,55
582,99,exam7,75


In [ ]:
df_students

,StudentID,First_Name,Last_Name,Degree
0,1,Pamela,Rodriguez,graduate
1,2,Kristie,Jackson,undergraduate
2,3,George,Curtis,graduate
3,4,Christie,Casey,undergraduate
4,5,Emily,Yoder,undergraduate
...,...,...,...,...
95,96,Benjamin,Dyer,undergraduate
96,97,Jason,Delgado,undergraduate
97,98,Kenneth,Cortez,undergraduate
98,99,Elizabeth,Short,undergraduate


In [ ]:
df = pd.merge(df_studentexamscores, df_students, on='StudentID')
df


# Group by student and exam, and compute the mean degree
df_averages = df.groupby(['StudentID', 'Exam'])['Degree'].mean().reset_index()
df_averages


TypeError: Could not convert graduate to numeric

In [ ]:
df_studentexamscores

,StudentID,Exam,Score
0,1,exam7,61
1,1,exam9,38
2,1,exam3,85
3,1,exam7,70
4,1,exam5,44
...,...,...,...
579,99,exam4,60
580,99,exam10,48
581,99,exam4,55
582,99,exam7,75


In [ ]:
df = pd.merge(df_studentexamscores, df_exams, on='Exam')

# Group by student and exam, and compute the mean score
df_averages = df.groupby(['Exam', 'Year'])['Score'].mean().round(2).reset_index()
# Sort the dataframe in descending order by the average score
df_averages.sort_values(by='Score', ascending=False, inplace=True)
df_averages["Score"].astype("int32")
df_averages["Year"] = df_averages["Year"].astype("int32")

df_averages = df_averages.rename(columns={'Score': 'average'})
df_averages.set_index('Exam', inplace=True)
df_averages

,Year,average
Exam,,
exam3,2018,86.16
exam8,2018,80.18
exam1,2016,75.09
exam7,2017,68.96
exam2,2017,64.28
exam6,2017,61.80
exam4,2019,54.04
exam5,2020,45.56
exam10,2019,44.68


In [ ]:
df = (
    pd.merge(df_studentexamscores, df_students, on = 'StudentID')
    .groupby(['Degree'])
    .mean()
    .round(2)
    .rename(columns = {'Score': 'Average'})
    )

df['Average'].to_frame()


,Average
Degree,
graduate,62.50
undergraduate,61.63


In [ ]:
temp_df = (
    pd.merge(df_studentexamscores, df_students, on = 'StudentID')
    .groupby(['StudentID'])
    .mean()
    .round(2)
    .reset_index()
    .sort_values(by = 'Score', ascending = False)
    .rename(columns = {'Score': 'average'})
    )

df = pd.merge(temp_df, df_students, on = 'StudentID')
df[['First_Name', 'Last_Name', 'Degree','average']].head(10)
df


,StudentID,average,First_Name,Last_Name,Degree
0,95,83.00,John,Washington,graduate
1,100,82.00,Robert,Andrade,undergraduate
2,18,79.00,Paul,Smith,undergraduate
3,97,77.50,Jason,Delgado,undergraduate
4,92,75.00,Calvin,Perez,undergraduate
...,...,...,...,...,...
95,85,47.33,William,Fowler,undergraduate
96,93,47.00,Helen,Davis,undergraduate
97,36,46.25,Samantha,Erickson,undergraduate
98,49,45.33,Eric,Schaefer,undergraduate


In [ ]:
df = pd.merge(df_studentexamscores, df_exams, on='Exam')

df_averages = (
    df
    .groupby(['Exam', 'Year'])[
    'Score']
    .mean()
    .round(2)
    .reset_index()
    )

df_averages.sort_values(by='Score', ascending=False, inplace=True)

df_averages["Year"] = df_averages["Year"].astype("int32")

df_averages = df_averages.rename(columns={'Score': 'average'})
df_averages.set_index('Exam', inplace=True)

df_averages

,Year,average
Exam,,
exam3,2018,86.16
exam8,2018,80.18
exam1,2016,75.09
exam7,2017,68.96
exam2,2017,64.28
exam6,2017,61.80
exam4,2019,54.04
exam5,2020,45.56
exam10,2019,44.68


## Part 2 Step 1


In [28]:
np.random.seed(0)
fake = Faker()
Faker.seed(0)
# ---- DO NOT CHANGE

# BEGIN SOLUTION
first_name_list = []
last_name_list = []
username_list = []

for i in range(100):
    fullname = fake.name().split(" ", 1)
    random_digits = str(np.random.randint(1000,9999))
    first_name_list.append(fullname[0])
    last_name_list.append(fullname[1])
    ith_user = fullname[0].lower()[0:2]+random_digits
    #print(ith_user)
    username_list.append(ith_user)
    #print([np.random.randint(1000,9999) for i in range(4)])

dicto = {'username' : username_list, 'first_name': first_name_list , 'last_name': last_name_list}
df = pd.DataFrame.from_dict(dicto)
df

,username,first_name,last_name
0,no3732,Norma,Fisher
1,jo4264,Jorge,Sullivan
2,el5859,Elizabeth,Woods
3,su8891,Susan,Wagner
4,pe5373,Peter,Montgomery
...,...,...,...
95,te7922,Terry,Wells
96,va5182,Vanessa,Cooper
97,mi1307,Michael,Simmons
98,ni6302,Nicholas,Kline


In [24]:
fake.name()

'Margaret Jones'

## part2_step2

In [40]:
mu = [35, 75, 25, 45, 45, 75, 25, 45, 35]
sigma = [9, 15, 7, 10, 5, 20, 8, 9, 10]
max_score = [50, 100, 40, 60, 50, 100, 50, 60, 50]

# Generate the scores using a normal distribution
scores = np.random.normal(mu, sigma, size = (100,9))
scores = np.round(scores)
scores_clip = np.clip(scores,0,max_score)
series_maal = scores_clip.T

hw_list = []
exam_list = []
hasho = {}

for i in range(5):
    hw_list.append((f"Hw{i+1}"))
    if i <=3:
        exam_list.append((f"Exam{i+1}"))   

hw_exam_list = hw_list + exam_list
print(hw_exam_list)

for i in range(len(hw_exam_list)):
    hasho[hw_exam_list[i]] = series_maal[i]
hasho
    


df2 = pd.DataFrame.from_dict(hasho)
df2

[32. 32. 39. 43. 26. 43. 50. 33. 10. 40. 50. 46. 48. 17. 48. 47. 48. 43.
 42. 33. 32. 50. 30. 43. 50. 41. 19. 25. 50. 26. 21. 21. 30. 40. 40. 50.
 40. 47. 38. 36. 40. 40. 18. 31. 39. 35. 37. 47. 31. 39. 27. 43. 26. 34.
 34. 34. 37. 29. 30. 48. 44. 39. 47. 34. 39. 35. 43. 39. 45. 32. 40. 35.
 42. 45. 42. 34. 21. 33. 19. 35. 41. 33. 38. 42. 40. 44. 39. 36. 36. 46.
 35. 45. 32. 30. 27. 44. 22. 43. 41. 39.]


,Hw1,Hw2,Hw3,Hw4,Hw5,Exam1,Exam2,Exam3,Exam4
0,32.0,67.0,34.0,50.0,43.0,87.0,24.0,33.0,13.0
1,32.0,94.0,19.0,42.0,43.0,42.0,13.0,40.0,34.0
2,39.0,86.0,21.0,35.0,48.0,68.0,20.0,50.0,36.0
3,43.0,64.0,22.0,52.0,43.0,72.0,11.0,49.0,49.0
4,26.0,92.0,28.0,32.0,46.0,50.0,28.0,55.0,21.0
...,...,...,...,...,...,...,...,...,...
95,44.0,60.0,23.0,28.0,43.0,75.0,15.0,59.0,27.0
96,22.0,88.0,27.0,54.0,42.0,69.0,23.0,59.0,44.0
97,43.0,100.0,16.0,37.0,50.0,83.0,24.0,45.0,33.0
98,41.0,76.0,29.0,55.0,50.0,73.0,32.0,37.0,23.0


In [42]:
hw_list = []
exam_list = []
hasho = {}

for i in range(5):
    hw_list.append((f"Hw{i+1}"))
    if i <=3:
        exam_list.append((f"Exam{i+1}"))   

hw_exam_list = hw_list + exam_list
print(hw_exam_list)

for i in range(len(hw_exam_list)):
    hasho[hw_exam_list[i]] = series_maal[i]
hasho
    


['Hw1', 'Hw2', 'Hw3', 'Hw4', 'Hw5', 'Exam1', 'Exam2', 'Exam3', 'Exam4']


{'Hw1': array([32., 32., 39., 43., 26., 43., 50., 33., 10., 40., 50., 46., 48.,
        17., 48., 47., 48., 43., 42., 33., 32., 50., 30., 43., 50., 41.,
        19., 25., 50., 26., 21., 21., 30., 40., 40., 50., 40., 47., 38.,
        36., 40., 40., 18., 31., 39., 35., 37., 47., 31., 39., 27., 43.,
        26., 34., 34., 34., 37., 29., 30., 48., 44., 39., 47., 34., 39.,
        35., 43., 39., 45., 32., 40., 35., 42., 45., 42., 34., 21., 33.,
        19., 35., 41., 33., 38., 42., 40., 44., 39., 36., 36., 46., 35.,
        45., 32., 30., 27., 44., 22., 43., 41., 39.]),
 'Hw2': array([ 67.,  94.,  86.,  64.,  92.,  56.,  86.,  68.,  81.,  84.,  47.,
         73.,  74.,  81.,  59.,  56.,  68.,  89.,  64.,  86.,  69.,  88.,
         85.,  59.,  66.,  65.,  80.,  58.,  87.,  56., 100.,  91.,  82.,
         93.,  77.,  86.,  75.,  68.,  45.,  97.,  85.,  80.,  80.,  77.,
         75.,  63.,  73.,  78.,  54.,  78.,  72.,  41.,  71.,  87.,  58.,
         86.,  89.,  92.,  90.,  83.,  59.,  84., 

In [43]:
scores = np.random.normal(mu, sigma, size = (100,9))
print(scores[0])
scores = np.round(scores)
print(scores[0])

[38.68316396 61.94497981 30.21176627 48.16816796 41.29470612 70.65967987
 17.4779607  25.09445749 35.39209341]
[39. 62. 30. 48. 41. 71. 17. 25. 35.]


In [45]:
# ---- DO NOT CHANGE
np.random.seed(0)
# ---- DO NOT CHANGE

mu = [35, 75, 25, 45, 45, 75, 25, 45, 35]
sigma = [9, 15, 7, 10, 5, 20, 8, 9, 10]
max_score = [50, 100, 40, 60, 50, 100, 50, 60, 50]

scores = np.random.normal(mu, sigma, size=(100, 9))
scores = np.round(scores)
scores_clip = np.clip(scores, 0, max_score)
series_maal = scores_clip.T

hw_list = []
exam_list = []
hasho = {}

for i in range(5):
    hw_list.append((f"Hw{i+1}"))
    if i <= 3:
        exam_list.append((f"Exam{i+1}"))

hw_exam_list = hw_list + exam_list

for i in range(len(hw_exam_list)):
    hasho[hw_exam_list[i]] = series_maal[i]
hasho

df2_scores =  pd.DataFrame.from_dict(hasho)

In [46]:
mu = [35, 75, 25, 45, 45, 75, 25, 45, 35]
sigma = [9, 15, 7, 10, 5, 20, 8, 9, 10]
df = df2_scores.describe().T.drop(['count','min','25%','50%','75%','max'], axis = 1).round(2)
df['mean_theoretical'] = mu
df['std_theoretical'] = sigma
df['abs_mean_diff'] = round(abs(df['mean'] - df['mean_theoretical']), 2)
df['abs_std_diff'] = round(abs(df['std'] - df['std_theoretical']), 2)
df

,mean,std,mean_theoretical,std_theoretical,abs_mean_diff,abs_std_diff
Hw1,34.45,8.38,35,9,0.55,0.62
Hw2,74.60,14.96,75,15,0.40,0.04
Hw3,24.94,6.51,25,7,0.06,0.49
Hw4,44.71,9.58,45,10,0.29,0.42
Hw5,43.57,4.67,45,5,1.43,0.33
Exam1,71.13,17.05,75,20,3.87,2.95
Exam2,25.45,8.40,25,8,0.45,0.40
Exam3,44.93,8.65,45,9,0.07,0.35
Exam4,32.34,8.76,35,10,2.66,1.24


In [47]:
np.random.seed(0)
fake = Faker()
Faker.seed(0)
# ---- DO NOT CHANGE

# BEGIN SOLUTION
first_name_list = []
last_name_list = []
username_list = []

for i in range(100):
    fullname = fake.name().split(" ", 1)
    random_digits = str(np.random.randint(1000, 9999))
    first_name_list.append(fullname[0])
    last_name_list.append(fullname[1])
    username_list.append(fullname[0].lower()[0:2]+random_digits)

dicto = {
    'username': username_list,
    'first_name': first_name_list,
    'last_name': last_name_list
}

df2_students =  pd.DataFrame.from_dict(dicto)

In [57]:
max_score = [50, 100, 40, 60, 50, 100, 50, 60, 50]

for i in range(5):
    hw_list.append((f"'Hw{i+1}'"))
    if i <= 3:
        exam_list.append((f"'Exam{i+1}'"))

hw_exam_list = hw_list + exam_list
print(hw_exam_list[0])
for i in range(len(hw_exam_list)):
    df2_students[hw_exam_list[i]] = ((df2_scores[hw_exam_list[i]]/max_score[i])*100).round()
df2_students
    
df2_students['Hw1'] = ((df2_scores['Hw1']/max_score[0])*100).round()
df2_students['Hw2'] = ((df2_scores['Hw2']/max_score[1])*100).round()
df2_students['Hw3'] = ((df2_scores['Hw3']/max_score[2])*100).round()
df2_students['Hw4'] = ((df2_scores['Hw4']/max_score[3])*100).round()
df2_students['Hw5'] = ((df2_scores['Hw5']/max_score[4])*100).round()
df2_students['Exam1'] = ((df2_scores['Exam1']/max_score[5])*100).round()
df2_students['Exam2'] = ((df2_scores['Exam2']/max_score[6])*100).round()
df2_students['Exam3'] = ((df2_scores['Exam3']/max_score[7])*100).round()
df2_students['Exam4'] = ((df2_scores['Exam4']/max_score[8])*100).round()
df_students.round()

Hw1


IndexError: list index out of range

In [60]:
for i in range(5):
    hw_list.append((f"'Hw{i+1}'"))
    if i <= 3:
        exam_list.append((f"'Exam{i+1}'"))

hw_exam_list = hw_list + exam_list
print('+ hw_exam_list[0] + ')

+ hw_exam_list[0] + 


In [62]:
testDf = pd.DataFrame(hw_exam_list)
testDf

,0
0,Hw1
1,Hw2
2,Hw3
3,Hw4
4,Hw5
...,...
76,'Exam4'
77,'Exam1'
78,'Exam2'
79,'Exam3'


In [104]:
df = pd.read_csv('part2_step5-input.csv')
dfhw2 = df[df.iloc[:,4] == 'AI_ISSUE']
dfhw3 = df[df.iloc[:,5] == 'AI_ISSUE']
dfhw4 = df[df.iloc[:,6] == 'AI_ISSUE']
dfhw5 = df[df.iloc[:,7] == 'AI_ISSUE']
dfex1 = df[df.iloc[:,8] == 'AI_ISSUE']
dfex2 = df[df.iloc[:,9] == 'AI_ISSUE']
dfex3 = df[df.iloc[:,10] == 'AI_ISSUE']
dfex4 = df[df.iloc[:,11] == 'AI_ISSUE']
pd5 = pd.concat([dfhw2,dfhw3,dfhw4,dfhw5,dfex1,dfex2,dfex3,dfex4], axis=0)
pd5['index1'] = pd5.index
pd6 = pd5.groupby(pd5.columns.tolist(),as_index=False).size()
pd6.sort_values(by = 'index1',inplace = True)
#print(pd6)
pd6 = pd6.rename(columns ={'size':'AI_Count'})
pd6[['username','first_name','last_name','AI_Count']].reset_index(drop = True)


,username,first_name,last_name,AI_Count
0,el5859,Elizabeth,Woods,2
1,st4468,Stephanie,Sutton,1
2,bo3163,Bobby,Flores,1
3,ki8877,Kimberly,Maynard,1
4,ki1797,Kimberly,Myers,1
5,wh8456,Whitney,Stark,1
6,ja5187,Jason,Carroll,1
7,do5420,Douglas,Allen,1
8,jo9752,Joseph,Freeman,1
9,jo3610,Joshua,Carter,1


In [91]:

dfhw2 = df[ df["Hw2"] == 'AI_ISSUE']
dfhw2

,username,first_name,last_name,Hw1,Hw2,Hw3,Hw4,Hw5,Exam1,Exam2,Exam3,Exam4
2,el5859,Elizabeth,Woods,88.0,AI_ISSUE,82.0,73.0,98.0,97.0,AI_ISSUE,63.0,68.0


In [105]:
df_columns = list(df.columns)
list_of_dfs = []
for i in range(4, len(df_columns)):
    list_of_dfs.append(df[df[df_columns[i]] == "AI_ISSUE"])
final_df = pd.concat(list_of_dfs, axis=0)
final_df['index1'] = final_df.index
mega_final_df = final_df.groupby(final_df.columns.tolist(),as_index=False).size()
mega_final_df.sort_values(by = 'index1',inplace = True)
mega_final_df = mega_final_df.rename(columns ={'size':'AI_Count'})
mega_final_df[['username','first_name','last_name','AI_Count']].reset_index(drop = True)

,username,first_name,last_name,AI_Count
0,el5859,Elizabeth,Woods,2
1,st4468,Stephanie,Sutton,1
2,bo3163,Bobby,Flores,1
3,ki8877,Kimberly,Maynard,1
4,ki1797,Kimberly,Myers,1
5,wh8456,Whitney,Stark,1
6,ja5187,Jason,Carroll,1
7,do5420,Douglas,Allen,1
8,jo9752,Joseph,Freeman,1
9,jo3610,Joshua,Carter,1


In [76]:
df_columns = list(df.columns)
df_columns

for i in range(3, len(df_columns)):
    print(df_columns[i])
    df_columns[i] = df[df_columns[i]]
    
mask_ai = df[df[df_columns[3:]] == "AI_ISSUE"]
mask_ai


Hw1
Hw2
Hw3
Hw4
Hw5
Exam1
Exam2
Exam3
Exam4


,username,first_name,last_name,Hw1,Hw2,Hw3,Hw4,Hw5,Exam1,Exam2,Exam3,Exam4
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,AI_ISSUE,NaN,NaN,NaN,NaN,AI_ISSUE,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AI_ISSUE,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
masko = df["Hw2"].str.contains("AI_ISSUE")
df = df[masko]
df

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [100]:
df2_from_file = pd.read_csv("part2_step5.csv", index_col=0)
df2_from_file

,username,first_name,last_name,AI_Count
0,el5859,Elizabeth,Woods,2
1,st4468,Stephanie,Sutton,1
2,bo3163,Bobby,Flores,1
3,ki8877,Kimberly,Maynard,1
4,ki1797,Kimberly,Myers,1
5,wh8456,Whitney,Stark,1
6,ja5187,Jason,Carroll,1
7,do5420,Douglas,Allen,1
8,jo9752,Joseph,Freeman,1
9,jo3610,Joshua,Carter,1


In [103]:
from pandas.testing import assert_frame_equal

assert_frame_equal(final_df, pd5)